In [153]:
## you can use sklearn
#from sklearn.datasets import load_iris
import glob
import numpy as np
import csv
from Function import *
#from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot  as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score  # 準確率
def read_csv(filename):
    t = []
    csv_file = open(filename,'r')
    i =0
    for row in csv.reader(csv_file):
        # if (i > 2):
        t.append(row)
        # i+=1
    # print (t)
    t = np.array(t,dtype = np.float32)
    return t

def accF (x,y):
    acc = 0
    for i in range(len(x)):
        if x[i] == y[i]:
            acc +=1
    acc = acc / len(x)
    return acc 


##################################
## your Feature extractor
##################################

def feature_ex(x):
    t = []
    x = np.array(x)
    indexAx = 0
    indexAy = 1
    indexAz = 2
    indexGx = 3
    indexGy = 4
    indexGz = 5
    indexTotalAcc = 6
    indexTotalGyro = 7
    indexRoll = 8
    indexPitch = 9
    
    totalAcc = getTotalAxes(x[indexAx],x[indexAy],x[indexAz])
    totalGyro = getTotalAxes(x[indexGx],x[indexGy],x[indexGz])
    roll = getRoll(x[indexAx],x[indexAz])
    pitch = getPitch(x[indexAy],x[indexAz])

    processedKoalaData = np.ones((10,40))

    for i in range(6):
        processedKoalaData[i] = copy.deepcopy(x[i])
    processedKoalaData[6] = copy.deepcopy(totalAcc)
    processedKoalaData[7] = copy.deepcopy(totalGyro)
    processedKoalaData[8] = copy.deepcopy(roll)
    processedKoalaData[9] = copy.deepcopy(pitch)



    mean = getMean2D(processedKoalaData)

    t.append(mean)


    return t




In [ ]:
sensor = []
label = []
feature = []
f = glob.glob(r'40_data/down'+'/*.csv')
for i in range(len(f)):
    t = read_csv(f[i])
    if (len(t[0])) == 40:
        t = feature_ex(t)
        sensor.append(t)
        label.extend([0])

f = glob.glob(r'40_data/up'+'/*.csv')
for i in range(len(f)):
    t = read_csv(f[i])
    if (len(t[0])) == 40:
        t = feature_ex(t)
        sensor.append(t)
        label.extend([0])

f = glob.glob(r'40_data/left'+'/*.csv')
for i in range(len(f)):
    t = read_csv(f[i])
    if (len(t[0])) == 40:
        t = feature_ex(t)
        sensor.append(t)
        label.extend([1])
f = glob.glob(r'40_data/right'+'/*.csv')
for i in range(len(f)):
    t = read_csv(f[i])
    if (len(t[0])) == 40:
        t = feature_ex(t)
        sensor.append(t)
        label.extend([1])

f = glob.glob(r'40_data/CW'+'/*.csv')
for i in range(len(f)):
    t = read_csv(f[i])
    if (len(t[0])) == 40:
        t = feature_ex(t)
        sensor.append(t)
        label.extend([2])

f = glob.glob(r'40_data/CCW'+'/*.csv')
for i in range(len(f)):
    t = read_csv(f[i])
    if (len(t[0])) == 40:
        t = feature_ex(t)
        sensor.append(t)
        label.extend([3])



f = glob.glob(r'40_data/VLR'+'/*.csv')
for i in range(len(f)):
    t = read_csv(f[i])
    if (len(t[0])) == 40:
        t = feature_ex(t)
        sensor.append(t)
        label.extend([4])

f = glob.glob(r'40_data/VRL'+'/*.csv')
for i in range(len(f)):
    t = read_csv(f[i])
    if (len(t[0])) == 40:
        t = feature_ex(t)
        sensor.append(t)
        label.extend([5])


f = glob.glob(r'40_data/non'+'/*.csv')
for i in range(len(f)):
    t = read_csv(f[i])
    if (len(t[0])) == 40:
        t = feature_ex(t)
        sensor.append(t)
        label.extend([6])

f = glob.glob(r'40_data/CRCW'+'/*.csv')
for i in range(len(f)):
    t = read_csv(f[i])
    if (len(t[0])) == 40:
        t = feature_ex(t)
        sensor.append(t)
        label.extend([7])

f = glob.glob(r'40_data/CRCCW'+'/*.csv')
for i in range(len(f)):
    t = read_csv(f[i])
    if (len(t[0])) == 40:
        t = feature_ex(t)
        sensor.append(t)
        label.extend([8])


In [ ]:
sensor = np.array(sensor)
print ('sensor shape is :',sensor.shape)

feature_name = [ "x["+str(i)+"]" for i in range((sensor.shape[1]*sensor.shape[2]))]
sensor = np.reshape(sensor,(sensor.shape[0],sensor.shape[1]*sensor.shape[2]))
label  = np.array(label)

print ('sensor shape after is :',sensor.shape)
print ('label shape is :',label.shape)
print ('label is :',label)

#fea=[ 0 , 1,  2,  3,  4,  7,  9]
#sensor=sensor.take(fea,axis = 1)

train_X, val_X, train_y, val_y =train_test_split(sensor, label, test_size = 0.1)


In [ ]:
#sc=StandardScaler()
#sc.fit(train_X)
#train_X=sc.transform(train_X)
#sc.fit(val_X)
#val_X=sc.transform(val_X)

In [ ]:
params = {
    'booster':'gbtree',
    'objective':'multi:softmax',
    'num_class':9,
    'gamma':0.1,
    'max_depth':11,
    'lambda':2,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'min_child_weight':3,
    'slient':1,
    'eta':0.1,
    'seed':1000,
    'nthread':4,
}
 
plst = list(params.items())
dtrain = xgb.DMatrix(train_X,train_y)
num_rounds = 500
# xgboost模型訓練
model = xgb.train(plst,dtrain,num_rounds)
dtest = xgb.DMatrix(val_X)
y_pred = model.predict(dtest)
 
# 計算準確率
accuracy = accuracy_score(val_y,y_pred)
print()
print('accuarcy:%.2f%%'%(accuracy*100))

In [151]:
f = glob.glob(r'testdata'+'/*.csv')
test_data=[None]*105
for i in range(len(f)):
    test = read_csv(f[i])
    s=f[i]
    s=s.strip('testdata/')
    s=s.strip('.csv')
   # print(s)
    if (len(test[0])) == 40:
        test = feature_ex(test)
        test_data[int(s)-1]=test
test_data=np.array(test_data)
test_data = np.reshape(test_data,(test_data.shape[0],test_data.shape[1]*test_data.shape[2]))
#test_data=test_data.take(fea,axis = 1)
print('test_data',test_data.shape)

dtest = xgb.DMatrix(test_data)
predict = model.predict(dtest)
print(predict.shape)
predict=predict.astype(int)
#predict[:10]

test_data (105, 10)
(105,)


In [152]:

with open('predict_0319.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Id','Category'])
    c=1
    for row in predict:
        #print([str(c).zfill(2)+".csv", row])
        writer.writerow([str(c).zfill(2)+".csv", row])
        c+=1
      # print_leaf(classify(row, my_tree))))